**Post-Processing Amazon Textract with Location-Aware Transformers**

# Part 3: Implementing Human Review

> *This notebook works well with the `Data Science 3.0 (Python 3)` kernel on SageMaker Studio - use the same as for NB1*

In this final notebook, we'll set up the human review component of the OCR pipeline using [Amazon Augmented AI (A2I)](https://aws.amazon.com/augmented-ai/): Completing the demo pipeline.

The A2I service shares a lot in common with SageMaker Ground Truth, with the main difference that A2I is designed for **near-real-time, single-example** annotation/review to support a live business process, while SMGT is oriented towards **offline, batch** annotation for building datasets.

The two services both use the Liquid HTML templating language, and you might reasonably wonder: "*Are we going to use the same custom boxes-plus-review template from earlier?*"

In fact, no we won't - for reasons we'll get to in a moment.

First though, let's load the required libraries and configuration for the notebook as before:

## Dependencies and configuration

The custom task template demonstrated in this notebook is a little more complex than the SageMaker Ground Truth one we saw in notebook 1, so is built with a [NodeJS](https://nodejs.org/en/)-based **toolchain** rather than edited as a raw HTML file.

- If you're running this notebook in SageMaker Studio, you can install NodeJS by running the below.
- If you're on a SageMaker Notebook Instance, check as it may already be installed - in which case you can skip this step.
- If you're running on some other environment (like a local laptop), you probably want to install NodeJS via standard tools instead. [nvm](https://github.com/nvm-sh/nvm) is a helpful utility for managing multiple different Node versions on your system.
- If you're not able to install NodeJS on your environment, don't worry - there's an alternative pre-built option (missing some features) mentioned later when we use it.

In [ ]:
# Check if NodeJS installed:
!node --version

In [ ]:
# Install NodeJS:
NODE_VER = "v18.15.0"
NODE_DISTRO = "linux-x64"
!mkdir -p /usr/local/lib/nodejs
!wget -c https://nodejs.org/dist/{NODE_VER}/node-{NODE_VER}-{NODE_DISTRO}.tar.xz -O - | tar -xJ -C /usr/local/lib/nodejs
# Can't easily override PATH here, so instead we'll just symlink relevant executable files into a
# folder that's already on the PATH:
NODE_BIN_DIR = f"/usr/local/lib/nodejs/node-{NODE_VER}-{NODE_DISTRO}/bin"
ONPATH_BIN_DIR = "/usr/local/bin"
!ln -fs {NODE_BIN_DIR}/node {ONPATH_BIN_DIR}/node && \
    ln -fs {NODE_BIN_DIR}/npm {ONPATH_BIN_DIR}/npm && \
    ln -fs {NODE_BIN_DIR}/npx {ONPATH_BIN_DIR}/npx && \
    echo "NodeJS {NODE_VER} installed!"

As before, once required libraries are installed, we can proceed with other imports and configuration:

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import json
from logging import getLogger
import os

# External Dependencies:
import boto3  # AWS SDK for Python
import sagemaker  # High-level SDK for SageMaker

# Local Dependencies:
import util

logger = getLogger()
role = sagemaker.get_execution_role()
s3 = boto3.resource("s3")
smclient = boto3.client("sagemaker")

# Manual configuration (check this matches notebook 1):
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = "textract-transformers/"
print(f"Working in bucket s3://{bucket_name}/{bucket_prefix}")
config = util.project.init("ocr-transformers-demo")
print(config)

# Field configuration saved from first notebook:
with open("data/field-config.json", "r") as f:
    fields = [
        util.postproc.config.FieldConfiguration.from_dict(cfg)
        for cfg in json.loads(f.read())
    ]
entity_classes = [f.name for f in fields]

# S3 URIs per first notebook:
raw_s3uri = f"s3://{bucket_name}/{bucket_prefix}data/raw"

## The rationale for a separate review template

For many ML-powered processes, intercepting low-confidence predictions for human review is important for delivering efficient, accurate service.

To deliver high-performing ML models sustainably, continuous collection of feedback for re-training is also important.

In this section we'll detail some reasons **why**; although joining the two processes together might be ideal; this example will demonstrate a **separate prediction review workflow** from the training data collection.

### Tension between process execution and model improvement

As we saw when setting up the pipeline in the last notebook, there's a **post-processing step** after the ML model - whose purpose is:

- To consolidate consecutive `WORD`s of the same class into a single "entity" detection via a simple heuristic
- To apply (configurable) business rules to consolidate entity detections into "fields" on the document (e.g. selecting a single value from multiple possible matches, etc).

Both of these processes are implemented in a simple Python Lambda function, and so would be technically straightforward to port into the ML model endpoint itself (in [src/inference.py](src/inference.py)). However, it's the **second one** that's important.

For any use case where there's a non-trivial **gap** between what the model is trained to estimate and what the business process consumes, there's a **tension** in the review process:

1. Reviewing business process fields is efficient, but does not collect model training data (although it may help us understand overall accuracy)
2. Reviewing the model inputs & outputs directly collects training data, but:
    - Does not directly review the accuracy of the end-to-end business process, so requires complete trust in the post-processing rules
    - May be inefficient, as the reviewer needs to collect more information than the business process absolutely requires (e.g. having to highlight every instance of `Provider Name` in the doc, when the business process just needs to know what the name is)
3. Splitting the review into multiple stages collects training/accuracy data for both components (ML model and business rules), but requires even more time - especially if the hand-off between the review stages might be asynchronous

In many cases the efficiency of the live process is most important for customer experience and cost management, and so approach 1 is taken (as we will in this example): With collection of additional model training data handled as an additional background task.

In some cases it may be possible to fully close the gap to resolve the tension and make a single offline-annotation/online-review UI work for everybody... E.g. for the credit cards use case, we might be able to:

- (Add effort) Move from word classification to a **sequence-to-sequence model**, to support more complex output processing (like OCR error correction, field format standardization, grouping words into matches, etc)... *OR*
- (Reduce scope) **Focus only on use-cases** where:
    - Each entity class only appears once in the document, *OR* most/every detection of multiple entities is equally important to the business process (may often be the case! E.g. on forms or other well-structured use cases) *AND*
    - Errors in OCR transcription or the heuristic rules to group matched words together are rare enough *or unpredictable enough* that there's no value in a confidence-score-based review (E.g. if "The OCR/business rules aren't making mistakes very often, and even when they do the confidence is still high - so our online review isn't helping these issues")

### A small techical challenge

So what if your use case for this model is:

- Seeing **high enough OCR accuracy rates** from Textract, and
- Enjoying good success with the heuristic for **joining classified words together** into multi-word entities based on the order Textract returned them, and
- Either having only **one match per entity type** per document; or where it's important to **always return multiple matches** if they're present?

Then maybe it would make sense roll your online review and training data collection into one process! By simply trusting the post-processing logic / OCR quality, and having reviewers use the bounding box tool.

**However,** there's one final hurdle: At the time of writing, the Ground Truth/A2I bounding box annotator works only for individual images, not PDFs. This means you'd also need to either:

- Restrict the pipeline to processing single-page documents/images, or
- Implement a custom box task UI capable of working with PDFs also, or
- Orchestrate around the problem by splitting and dispatching each document to multiple single-page A2I reviews.

### In summary

For some use cases of technology like this, directly using the training data annotation UI for online review could be the most efficient option.

But to avoid ignoring the (potentially large) set of viable use cases where it's not practical; and to avoid introducing complexity or workarounds for handling multi-page documents; this sample presents a separate tailored online review UI.


## Develop the review task template

Just as with SageMaker Ground Truth, a custom task UI template has been provided and we can preview it via the SageMaker SDK.

The interfaces for building SMGT and A2I templates are generally very similar but in this particular case there are some differences here from our notebook 1:

1. This template accepts the list of fields dynamically at run-time, so **no extra parameter substitutions** are required in the template itself
1. The input to this stage of the pipeline is a little more complex than a simple image + Amazon Textract result URI: So we'll use an **example JSON file** and substitute the Textract URI to match your bucket and prefix (so the displayed file will not match the extracted field content)
1. Since the custom template here is a little more complex, we use a **NodeJS-based toolchain** to build it rather than directly authoring a browser-ready HTML file. You can find more detailed information about the reasons and practicalities for this in the [review/README.md](review/README.md) file.

First, you'll need to set up the custom UI project in the `review/` folder - installing the additional dependencies it requires:

> ⚠️ **If you have problems** with the node/npm build process, first try re-running the cell. In some cases we've seen intermittent permissions errors that can be resolved by retrying.
>
> If it still won't work, you can instead fall back to the provided legacy straight-to-HTML template instead - by setting:
>
> ```python
> ui_template_file = "review/fields-validation-legacy.liquid.html"  # (Already exists)
> ```

In [ ]:
!cd review && npm install

Then, build the UI HTML template from source:

In [ ]:
!cd review && npm run build
ui_template_file = "review/dist/index.html"

Next, prepare the dummy task JSON object for usefully previewing the UI:

In [ ]:
# Load the sample input from file:
with open("review/task-input.example.json", "r") as f:
    sample_obj = json.loads(f.read())

# Find any `a_pdf_s3uri`, so long as it exists in your account:
textract_s3key_root = f"{bucket_prefix}data/raw"
try:
    a_pdf_s3obj = next(filter(
        lambda o: o.key.endswith(".pdf"),
        s3.Bucket(bucket_name).objects.filter(Prefix=textract_s3key_root)
    ))
    a_pdf_s3uri = f"s3://{a_pdf_s3obj.bucket_name}/{a_pdf_s3obj.key}"
except StopIteration as e:
    raise ValueError(
        f"Couldn't find any .pdf files in s3://{bucket_name}/{textract_s3key_root}"
    ) from e

# Substitute the PDF URI in the sample input object:
sample_obj["TaskObject"] = a_pdf_s3uri

Finally, render the template:

In [ ]:
ui_render_file = "review/render.tmp.html"
with open(ui_template_file, "r") as fui:
    with open(ui_render_file, "w") as frender:
        ui_render_resp = smclient.render_ui_template(
            UiTemplate={ "Content": fui.read() },
            Task={ "Input": json.dumps(sample_obj) },
            RoleArn=role,
        )
        frender.write(ui_render_resp["RenderedContent"])

if "Errors" in ui_render_resp:
    if (ui_render_resp["Errors"] and len(ui_render_resp["Errors"])):
        print(ui_render_resp["Errors"])
        raise ValueError("Template render returned errors")

print(f"▶️ Open {ui_render_file} and click 'Trust HTML' to see the UI in action!")

Opening [review/render.tmp.html](review/render.tmp.html) and clicking **Trust HTML** in the toolbar, you should see a view something similar to the below.

In this UI, the model's detections are rendered as bounding boxes over the source document with the same class colours as the original annotation view. In the right panel, you can view and amend the detected values for each field or use the checkboxes to toggle whether the field is present in the document or not. Both single- and multi-value fields are supported, and the overall confidence of detection is shown as a bar graph for each field type.

![](img/a2i-custom-template-demo.png "Screenshot of custom review UI")

## Set up the human review workflow

Similarly to a SageMaker Ground Truth labelling job, we have 3 main concerns for setting up an A2I review workflow:

- **Who's** doing the labelling
- **What** the task will look like
- **Where** the output reviews will be stored to once the review completes (i.e. location on Amazon S3)

Our **workteam** from notebook 1 should already be set up.

▶️ **Check** the workteam name below matches your setup, and run the cell to store the ARN:

In [ ]:
workteam_name = "just-me"  # TODO: Update this to match yours, if different

workteam_arn = util.smgt.workteam_arn_from_name(workteam_name)

Our **template** has been tested as above, so just needs to be registered with A2I.

You can use the below code to register your template and store its ARN, but can also refer to the [A2I Console worker task templates list](https://console.aws.amazon.com/a2i/home?#/worker-task-templates)

In [ ]:
with open(ui_template_file, "r") as f:
    create_template_resp = smclient.create_human_task_ui(
        HumanTaskUiName="fields-validation-1",  # (Can change this name as you like)
        UiTemplate={"Content": f.read()},
    )

task_template_arn = create_template_resp["HumanTaskUiArn"]
print(f"Created A2I task template:\n{task_template_arn}")

To finish setting up the "workflow" itself, we need 2 more pieces of information:

- The **location in S3** where review outputs should be stored
- An appropriate **execution role** which will give the A2I workflow to read input documents and write review results.

These are determined by the **OCR pipeline solution stack**, because the reviews bucket is created by the pipeline with event triggers to resume the next stage when reviews are uploaded.

The code below should be able to look up these parameters for you automatically:

In [ ]:
reviews_bucket_name = config.pipeline_reviews_bucket_name
print(reviews_bucket_name)
reviews_role_arn = config.a2i_execution_role_arn
print(reviews_role_arn)

Alternatively, you may **find** your pipeline solution stack from the [AWS CloudFormation Console](https://console.aws.amazon.com/cloudformation/home?#/stacks) and click through to the stack detail page. From the **Outputs** tab, you should see the `A2IHumanReviewBucketName` and `A2IHumanReviewExecutionRoleArn` values as shown below.

(You may also note the `A2IHumanReviewFlowParamName`, which we'll use in the next section)

![](img/cfn-stack-outputs-a2i.png "CloudFormation stack outputs for OCR pipeline")

Once these values are populated, you're ready to create your review workflow by running the code below.

Note that you can also manage flows via the [A2I Human Review Workflows Console](https://console.aws.amazon.com/a2i/home?#/human-review-workflows/).

In [ ]:
create_flow_resp = smclient.create_flow_definition(
    FlowDefinitionName="ocr-fields-validation-1",  # (Can change this name as you like)
    HumanLoopConfig={
        "WorkteamArn": workteam_arn,
        "HumanTaskUiArn": task_template_arn,
        "TaskTitle": "Review OCR Field Extractions",
        "TaskDescription": "Review and correct credit card agreement field extractions",
        "TaskCount": 1,  # One reviewer per item
        "TaskAvailabilityLifetimeInSeconds": 60 * 60,  # Availability timeout
        "TaskTimeLimitInSeconds": 60 * 60,  # Working timeout
    },
    OutputConfig={
        "S3OutputPath": f"s3://{reviews_bucket_name}/reviews",
    },
    RoleArn=reviews_role_arn,
)

print(f"Created review workflow:\n{create_flow_resp['FlowDefinitionArn']}")

## Integrate with the OCR pipeline

Once the human review workflow is created, the final integration step is to point the pipeline at it - just as we did for our SageMaker endpoint earlier.

In code, this can be done as follows:

In [ ]:
print(f"Configuring pipeline with review workflow: {create_flow_resp['FlowDefinitionArn']}")

ssm = boto3.client("ssm")
ssm.put_parameter(
    Name=config.a2i_review_flow_arn_param,
    Overwrite=True,
    Value=create_flow_resp["FlowDefinitionArn"],
)

Alternatively through the console, you would follow these steps:

▶️ **Check** the `A2IHumanReviewFlowParamName` output of your OCR pipeline stack in [CloudFormation](https://console.aws.amazon.com/cloudformation/home?#/stacks) (as we did above)

▶️ **Open** the [AWS Systems Manager Parameter Store console](https://console.aws.amazon.com/systems-manager/parameters/?tab=Table) and **find the review flow parameter in the list**.

▶️ **Click** on the name of the parameter to open its detail page, and then on the **Edit** button in the top right corner. Set the value to the **workflow ARN** (see previous code cell in this notebook) and save the changes.

![](img/ssm-a2i-param-detail.png "Screenshot of SSM parameter detail page for human workflow")

## Final testing

Your OCR pipeline should now be fully functional! Let's try it out:

▶️ **Log in** to the labelling portal (URL available from the [SageMaker Ground Truth Workforces Console](https://console.aws.amazon.com/sagemaker/groundtruth?#/labeling-workforces) for your correct AWS Region)

![](img/smgt-find-workforce-url.png "Screenshot of SMGT console with workforce login URL")

▶️ **Upload** one of the sample documents to your pipeline's input bucket in Amazon S3, either using the code snippets below or drag and drop in the [Amazon S3 Console](https://console.aws.amazon.com/s3/)

In [ ]:
pdfpaths = []
for currpath, dirs, files in os.walk("data/raw"):
    if "/." in currpath or "__" in currpath:
        continue
    pdfpaths += [
        os.path.join(currpath, f) for f in files
        if f.lower().endswith(".pdf")
    ]
pdfpaths = sorted(pdfpaths)

In [ ]:
test_filepath = pdfpaths[14]
test_s3uri = f"s3://{config.pipeline_input_bucket_name}/{test_filepath}"

!aws s3 cp '{test_filepath}' '{test_s3uri}'

▶️ **Open up** your "Processing Pipeline" state machine in the [AWS Step Functions Console](https://console.aws.amazon.com/states/home?#/statemachines)

After a few seconds you should find that a Step Function execution is automatically triggered and (since we enabled so many fields that at least one is always missing) the example is eventually forwarded for human review in A2I.

As you'll see from the `ModelResult` field in your final *Step Output*, this pipeline produces a rich but usefully-structured output - with good opportunities for onward integration into further Step Functions steps or external systems. You can find more information and sample solutions for integrating AWS Step Functions in the [Step Functions Developer Guide](https://docs.aws.amazon.com/step-functions/latest/dg/welcome.html).

![](img/sfn-statemachine-success.png "Screenshot of successful Step Function execution with output JSON")

## Conclusion

In this worked example we showed how advanced, open-source language processing models specifically tailored for document understanding can be integrated with [Amazon Textract](https://aws.amazon.com/textract/): providing a trainable, ML-driven framework for tackling more niche or complex requirements where Textract's [built-in structure extraction tools](https://aws.amazon.com/textract/features/) may not fully solve the challenges out-of-the-box.

The underlying principle of the model - augmenting multi-task neural text processing architectures with positional data - is highly extensible, with potential to tackle a wide range of use cases where joint understanding of the content and presentation of text can deliver better results than considering text alone.

We demonstrated how an end-to-end process automation pipeline applying this technology might look: Developing and deploying the model with [Amazon SageMaker](https://aws.amazon.com/sagemaker/), building a serverless workflow with [AWS Step Functions](https://aws.amazon.com/step-functions/) and [AWS Lambda](https://aws.amazon.com/lambda/), and driving quality with human review of low-confidence documents through [Amazon Augmented AI](https://aws.amazon.com/augmented-ai/).

Thanks for following along, and for more information, don't forget to check out:

- The other published [Amazon Textract Examples](https://docs.aws.amazon.com/textract/latest/dg/other-examples.html) listed in the [Textract Developer Guide](https://docs.aws.amazon.com/textract/latest/dg/what-is.html)
- The extensive repository of [Amazon SageMaker Examples](https://github.com/aws/amazon-sagemaker-examples) and usage documentation in the [SageMaker Python SDK User Guide](https://sagemaker.readthedocs.io/en/stable/) - as well as the [SageMaker Developer Guide](https://docs.aws.amazon.com/sagemaker/index.html)
- The wide range of other open algorithms and models published by [HuggingFace Transformers](https://huggingface.co/transformers/), and their specific documentation on [using the library with SageMaker](https://huggingface.co/transformers/sagemaker.html)
- The conversational AI and NLP area (and others) of Amazon's own [Amazon.Science](https://www.amazon.science/conversational-ai-natural-language-processing) blog

Happy building!